<a href="https://colab.research.google.com/github/Hyperdraw/ai-workshop-project/blob/dev/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Workshop Project

In [ ]:
# This notebook will contain the main code for the project

Sequential model

In [6]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
def cur_model(num_Conv2D,num_filters,num_kernel):
  model = Sequential()
  for i in range(num_Conv2D):
    model.add(Conv2D(num_filters, num_kernel, activation ='relu'))
